In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# colab 사용시
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
# colab 용
# os.chdir('/content/drive/MyDrive')
# local 용
os.chdir('./')
!ls

2014732040_김병준_midterm.ipynb  test_data  train_data


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

cpu


In [5]:
trans = transforms.Compose([
                            transforms.Resize((256,340)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
train_data = datasets.ImageFolder(root='train_data/', transform=trans)

In [6]:
# 데이터 확인용 코드
# for num, data in enumerate(train_data):
#     sample, label = data
#     print(num, sample, label)
# 데이터 확인 결과 input=3(RGB) size=(256x340) label=3

In [7]:
batch_size = 128
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer5 = nn.Sequential(
            nn.Linear(16*21*256, 120),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc = nn.Linear(120, 3)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        print(out.shape) # 출력 size 확인용 코드
        out = out.view(out.size(0), -1)
        out = self.layer5(out)
        out = self.fc(out)
        return out

In [11]:
model = CNN().to(device)

# 임의 value로 모델 확인
value = torch.Tensor(1,3,256,340)
print(model(value).shape)

torch.Size([1, 256, 16, 21])
torch.Size([1, 3])


In [12]:
learning_rate = 0.003
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(train_dataloader)
epochs = 15

for epoch in range(epochs):
    avg_cost = 0

    for X_train, Y_train in train_dataloader:
        X_train = X_train.to(device)
        Y_train = Y_train.to(device)

        hypo = model(X_train)
        cost = criterion(hypo, Y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print(f'Epoch {epoch+1}/{epochs} Cost: {avg_cost:.6f}')

In [ ]:
test_data = datasets.ImageFolder(root='test_data/', transform=trans)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

In [ ]:
with torch.no_grad():
    for X_test, Y_test in test_dataloader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()

    print(f'Accuracy: {accuracy*100:2.2f}%')